In [1]:
import IPython.core.display as di

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) '
                '{ jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>',
                raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)


Toggle code

In [2]:
import glob
import os.path as op

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from scipy.signal import detrend
from scipy.stats import trim_mean
from scipy.stats import kendalltau, pearsonr

from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

import mne
from mne.utils import _time_mask as freq_mask
from mne.externals.h5io import read_hdf5, write_hdf5

from library.stats import compute_corr
from library.downloaders import get_single_trial_source_psd
from library.stats import compute_log_linear_fit

In [3]:
mne.utils.set_log_level('warning')

In [4]:
%reload_ext autoreload
%autoreload 2

### loading source data

In [5]:
# %run hcp_compute_subject.py --subject 100307 --s3 --downloaders \
#     "/home/ubuntu/github/swish/aws_scripts/library:downloaders.get_single_trial_source_psd" --keep_files \
#     --hcp_no_meg --hcp_no_anat

In [6]:
# %run hcp_compute_subject.py --subject 100307 --s3 --downloaders \
#     "/home/ubuntu/github/swish/aws_scripts/library:downloaders.get_psds_times" --keep_files \
#     --hcp_no_meg --hcp_no_anat

### get all file names and group by run

In [7]:
# stc_files = dict(r0=list(), r1=list(), r2=list())
# for fname in glob.glob('/mnt/hcp-meg/100307/*stc'):
#     for pattern in get_single_trial_source_psd('100307')['key_list']:
#         if glob.fnmatch.fnmatch(fname, '*' + pattern):
#             if 'r1' in fname:
#                 key = 'r1'
#             elif 'r2' in fname:
#                 key = 'r2'
#             else:
#                 key = 'r0'
#             stc_files[key].append(fname)

## compute grand average alpha

In [8]:
# X = 0.
# for ii, fnames in enumerate(sum(stc_files.values(), [])):
#     stc = mne.read_source_estimate(fname)
#     X += np.log10(stc.data)
# stc._data = X
# stc._data /= ii

## compute grand average 1/f

In [9]:
# def stc_gen(stc_files):
#     for ii, fnames in enumerate(sum(stc_files.values(), [])):
#         stc = mne.read_source_estimate(fname)
#         yield stc.data

# coefs_, _, mse_, _ = compute_log_linear_fit(
#     stc_gen(stc_files), freqs=np.linspace(0.001, 127, len(stc.times)), sfmin=0.1, sfmax=1.,
#     log_fun=np.log10)

## compute label time courses for all runs

In [10]:
# from library.workflows import get_brodmann_labels
# written_files = get_brodmann_labels(spacing='oct5', subjects_dir='/home/ubuntu/freesurfer/subjects')

In [11]:
# if not op.exists('/mnt/hcp-subjects'):
#     os.mkdir('/mnt/hcp-subjects')

# if not op.exists('/mnt/hcp-subjects/fsaverage'):
#     os.symlink('/home/ubuntu/freesurfer/subjects/fsaverage',
#                '/mnt/hcp-subjects/fsaverage')
# [f for f in os.listdir('/mnt/hcp-subjects/fsaverage/label') if 'oct5' in f][:10]

In [12]:
# src_orig = mne.setup_source_space(subject='fsaverage', fname=None, spacing='oct5', add_dist=False,
#                                   subjects_dir='/mnt/hcp-subjects')

In [13]:
# label_tcs = list()
# for ii, fname in enumerate(sum(stc_files.values(), [])):
#     stc = mne.read_source_estimate(fname)
#     stc.subject = '100307'
#     stc = stc.to_original_src(src_orig, 'fsaverage', subjects_dir=hcp_subjects_dir)
#     label_tcs.append(
#         np.array(
#             [stc.extract_label_time_course(label, src_orig, mode='mean')
#              for label in labels]))

In [14]:
# label_tcs = np.array(label_tcs)[:, :, 0, :]
# print(label_tcs.shape)

In [15]:
# %matplotlib inline
# freqs = np.linspace(0.001, 127, len(stc.times))
# X = np.log10(label_tcs).std(0).T
# X_mean = X.mean(1)
# X_std = X.std(1)
# plt.plot(np.log10(freqs), X_mean, color='steelblue');
# plt.fill_between(np.log10(freqs), X_mean - X_std, X_mean + X_std, color='steelblue', alpha=0.2);
# plt.xticks(np.log10([0.1, 1, 10, 100]), [0.1, 1, 10, 100])
# plt.xlim(np.log10([0.1, 100]))

## compute grand average correlations on label time courses

In [16]:
df = pd.read_csv('unrelated_MEG_subjects.csv')
subjects_missing = pd.read_csv('missing_freesurfer_s900_unrelated.csv').SubjectId.tolist()
subjects = ' '.join([str(sub) for sub in df.Subject.tolist() if sub not in subjects_missing])

In [90]:
%%bash
rm -rf /mnt/*


rm: cannot remove ‘/mnt/swapfile’: Operation not permitted


In [91]:
ls /mnt

swapfile


In [92]:
%run hcp_compute_subject.py --subject 100307\
    --downloaders "/home/ubuntu/github/meg_study_dynamic_scales/scripts/library:downloaders.get_psds_times" \
    --s3 \
    --hcp_no_anat\
    --hcp_no_meg \
    --keep_files \
    --mkl_num_threads 1

Elapsed time downloading hcp-openaccess from s3 00:00:00
Elapsed time downloading hcp-meg-data from s3 00:00:00
calling "download_only./home/ubuntu/github/meg_study_dynamic_scales/scripts/hcp_compute_subject" with:
	subject: 100307
downloading only
uploading /mnt/hcp-meg/100307/download_only/2016-05-13_14-46-46/call.txt
uploading /mnt/hcp-meg/100307/download_only/2016-05-13_14-46-46/call.txt
uploading /mnt/hcp-meg/100307/download_only/2016-05-13_14-46-46/run_output.log
uploading /mnt/hcp-meg/100307/download_only/2016-05-13_14-46-46/run_time.json
uploading /mnt/hcp-meg/100307/download_only/2016-05-13_14-46-46/hcp_compute_subject.py
uploading /mnt/hcp-meg/100307/download_only/2016-05-13_14-46-46/script.py
uploading /mnt/hcp-meg/100307/download_only/2016-05-13_14-46-46/report.html
uploading /mnt/hcp-meg/100307/download_only/2016-05-13_14-46-46/done
uploading /mnt/hcp-meg/100307/download_only/2016-05-13_14-46-46/written_files.txt
Elapsed time uploading to s3 00:00:00
Elapsed time for runni

In [ ]:
%run hcp_compute_subject.py --subject 599671\
    --downloaders "/home/ubuntu/github/meg_study_dynamic_scales//library:downloaders.get_psds_times" \
                  "/home/ubuntu/github/meg_study_dynamic_scales//library:downloaders.get_single_trial_source_psd" \
    --s3 \
    --hcp_no_meg \
    --keep_files \
    --mkl_num_threads 1 \
    --fun_path "/home/ubuntu/github/meg_study_dynamic_scales/library:workflows.compute_source_outputs" \
    --fun_args --spacing oct5 --debug False

Elapsed time downloading hcp-openaccess from s3 00:00:02
Elapsed time downloading hcp-meg-data from s3 00:00:00


In [88]:
%%bash
rm output.txt
here=/home/ubuntu/github/meg_study_dynamic_scales/scripts
# python ./run_parallel.py --script  hcp_compute_subject.py --par_args 599671\
python ./run_parallel.py --script hcp_compute_subject.py --par_args 599671 665254 825048 877168 814649 912447 200109 662551 872764 898176 660951 568963 191841 153732 559053 917255 352738 715950 783462 680957 512835 177746 255639 601127 433839\
    --par_target subject --sleep 30 --n_par 8 --args \
    --downloaders "$here/library:downloaders.get_psds_times" \
                  "$here/library:downloaders.get_single_trial_source_psd" \
    --s3 \
    --hcp_no_meg \
    --mkl_num_threads 1 \
    --fun_path "$here/library:workflows.compute_source_outputs" \
    --fun_args --spacing oct5 --debug False &> output.txt

rm: cannot remove ‘output.txt’: No such file or directory


In [89]:
mne.SourceEstimate.to_original_src?